## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


* User-User предсказывает рейтинг каждого пользователя как некоторую линейную комбинацию рейтинга К соседей пользователей и товары являются признаками для пользователей. 

* Item-Item имеет такой же принцы, только ищет похожи соседей в K товаров, в данном случае пользователи являются признаками для товаров.

* Работать item_item_rec под user_user_rec  сможет после переустановки input/output данных и переобучению модели

* В основу item-item рекомендаций заложен алгоритм KNN "К-ближайших" соседей и находят похожих пользователей, а в ALS используется алгоритм Градиентного спуска и основа decomposition матрицы на 2 матрицы других размерностей

* в ALS можно ранжировать список рекомендаций в отличие от item-item рекомендаций

* АLS Прогнозирует вероятность, а Item-Item Recommender некоторые числа  (про вероятность в ALS вы говорили на 4 вебинаре когд сравнивали с Light FM!)


2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


* комментарии/отзывы и их количество,
* кол-во просмотров
* лояльность покупателей
* сезонность

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


* Плохо работает на "холодном" старте из-за невозможности работы с вешними данными (признаки товаров и пользователей)
* Не учитывается сезонность, нельзя использовать для рекомендаций сезонных товаров
* ALS учитывает только позитивные линейные взаимодействия факторов


4. Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?


* Первое. Вводится зависимость релевантности от вхождения или не вхождения слов в запросах с более чем одного слова. Релевантность всего поискового запроса равна сумме релевантностей всех слов. Таким образом, отсутствие слова или другими словами image (его частота) равна 0 дает релевантность 0.
* Второе. Преимущество при поиске в запросах с более чем 2-ух слов, одно из которых менее употребительно (более узкоспециализированное) будет отдаваться документам которые содержат это узкоспециализированное слово.

## Практическая часть


In [1]:
import implicit

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import time

In [3]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    precision = flags.sum()/len(recommended_list)
    
    return precision

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


## 1. ALS

In [4]:
data = pd.read_csv('../webinar_2/retail_train.csv')
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity.rename(columns={'sales_value': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,259120,397896,420647,480014,818981,819063,819255,819304,819308,819330,...,15926844,15926886,15971546,15972074,15972298,15972565,15972790,16053266,16100266,16769635
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
user_item_matrix.shape

(2499, 5001)

In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [10]:
%%time

model = AlternatingLeastSquares(factors=100, #k - matrix rank made out of independed rows
                                regularization=0.001, # lambda like ridge lasso
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 6.17 s


In [11]:
[id_to_itemid[rec[0]] for rec in recs]

[1133018, 1106523, 999999, 5569230, 1082185]

In [12]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [13]:
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [14]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 1min 11s


0.16356513222330837

### 2. TF-IDF взвешивание

In [15]:
user_item_matrix.T.head(2)  # before applying TF-IDF

user_id,1,2,3,4,5,6,7,8,9,10,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
item_id,,,,,,,,,,,,,,,,,,,,,
259120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
type(user_item_matrix.T)

pandas.core.frame.DataFrame

In [19]:
user_item_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

### Вопрос: раскажите на уроке по подробнее что делает tfidf_weight как я вижу он не только дабавляет веса но еще и меняет структуру

In [20]:
type(user_item_matrix)   # turned from a numpy array to a float

scipy.sparse.coo.coo_matrix

In [44]:
user_item_matrix.tocsr()[1:5,:].todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
print(user_item_matrix)

  (19, 0)	5.533081369892069
  (67, 0)	3.2797720401564856
  (984, 0)	4.885074701751093
  (1181, 0)	12.78415842856356
  (1246, 0)	3.9908034123029075
  (1419, 0)	5.497024045417256
  (1691, 0)	4.581497468143318
  (2143, 0)	8.95620357581946
  (2490, 0)	5.10402258742199
  (19, 1)	5.533081369892069
  (31, 1)	9.50052456419734
  (67, 1)	3.2797720401564856
  (114, 1)	3.388287049055229
  (130, 1)	4.123389190276642
  (192, 1)	5.175226602467874
  (211, 1)	5.735558984143008
  (255, 1)	4.131074980702863
  (324, 1)	4.744810157122492
  (350, 1)	3.7113171930807476
  (351, 1)	11.687541245309895
  (382, 1)	5.18558769068393
  (403, 1)	4.5195460951563815
  (485, 1)	17.708959350166545
  (517, 1)	3.680934126275075
  (525, 1)	4.920227945335344
  :	:
  (2019, 4999)	2.908024230291808
  (2112, 4999)	5.238128594907839
  (2234, 4999)	4.996573737124163
  (2266, 4999)	5.735573113125257
  (2273, 4999)	3.726690910207751
  (2281, 4999)	7.538252831679815
  (2298, 4999)	5.175226602467874
  (2311, 4999)	3.685964651709931
 

In [47]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))


Wall time: 1min 5s


0.17100881488736291

In [49]:
result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

0.17100881488736291

## 3. BM25 взвешивание

In [50]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,259120,397896,420647,480014,818981,819063,819255,819304,819308,819330,...,15926844,15926886,15971546,15972074,15972298,15972565,15972790,16053266,16100266,16769635
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [54]:
type(user_item_matrix)

scipy.sparse.coo.coo_matrix

In [56]:
print(user_item_matrix)

  (19, 0)	13.36898011975866
  (67, 0)	9.433713418797142
  (984, 0)	12.303661633531117
  (1181, 0)	94.31116232009599
  (1246, 0)	10.752112407525427
  (1419, 0)	13.310835937568456
  (1691, 0)	11.788499789001646
  (2143, 0)	18.430461517087796
  (2490, 0)	12.668600213898381
  (19, 1)	7.871597390611639
  (31, 1)	58.72937145559752
  (67, 1)	5.554529460435917
  (114, 1)	5.67638223590623
  (130, 1)	11.248439636758487
  (192, 1)	7.528429979137215
  (211, 1)	19.184575562721612
  (255, 1)	6.478287537612436
  (324, 1)	7.104999671941771
  (350, 1)	6.031660435926251
  (351, 1)	12.95886421871345
  (382, 1)	7.538474849779556
  (403, 1)	6.878304244512666
  (485, 1)	88.95130489214881
  (517, 1)	5.998696164198254
  (525, 1)	12.654535609732994
  :	:
  (2019, 4999)	6.022146265253229
  (2112, 4999)	8.915315624190042
  (2234, 4999)	8.63907980946925
  (2266, 4999)	9.47117437037693
  (2273, 4999)	7.105067391652432
  (2281, 4999)	11.364048673585337
  (2298, 4999)	8.843799049304181
  (2311, 4999)	7.0532086187608

In [57]:

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

In [65]:
result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

0.1917727717923576

In [61]:
result.head(3)

,user_id,actual,als,als_tfidf,als_bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 5569374, 878996, 979707, 995242]","[1033142, 999999, 1005186, 1041796, 1082185]","[999999, 1082185, 995242, 965766, 1100972]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 5568378, 1133018, 1092026, 910032]","[951590, 999999, 1042438, 1106523, 1082185]","[1133018, 999999, 1106523, 951590, 1053690]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1051516, 986912, 1007195, 904360, 1023720]","[1042697, 1024306, 878996, 834484, 1007195]","[1082185, 1098248, 1024306, 878996, 965267]"


In [70]:
pd.DataFrame(
    [
        result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()
              
    ], columns = ['precision_at_k'], index=[['als','als_bm25', 'als_tfidf']])

,precision_at_k
als,0.163565
als_bm25,0.191773
als_tfidf,0.171009


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.GridSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!